# 전문가를 위한 파이썬

## part 3

## 챕터6

### 전략 패턴의 리펙토링

* 고전적인 전략 : UML
    * 콘텍스트 : 일부 계산을 서로 다른 알고리즘을 구현하는 교환 가능한 컴포넌트에 위임함으로써 서비스를 제공한다.
    * 전략 : 여러 알고리즘을 수현하는 컴포넌트에 공통된 인터페이스.
    * 구체적인 전략 : 전력의 구상 서브클래스 중 하나.
* "디자인 패턴"에서는 전략 패턴을 다음과 같이 설명한다. : 
    - 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.

In [60]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): # the Strategy: an Abstract Base Class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""
        
class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [61]:
Joe=Customer('John Doe',0)
Ann=Customer('Ann Smith',1100)
cart=[LineItem('banana', 4, 0.5),
      LineItem('apple', 25, 1.5),
      LineItem('melon', 5, 5.0)]
print(Order(Joe, cart, BulkItemPromo()))
print(Order(Ann, cart, FidelityPromo()))

<Order total: 64.50 due: 60.75>
<Order total: 64.50 due: 61.27>


* Promotion을 추상 베이스 클래스로 구현했으므로, 전략 페턴을 더욱 명시적으로 보여준다.

In [65]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [66]:
Joe=Customer('John Doe',0)
Ann=Customer('Ann Smith',1100)
cart=[LineItem('banana', 4, 0.5),
      LineItem('apple', 25, 1.5),
      LineItem('melon', 5, 5.0)]
print(Order(Joe, cart, bulk_item_promo))
print(Order(Ann, cart, fidelity_promo))

<Order total: 64.50 due: 60.75>
<Order total: 64.50 due: 61.27>


* 좀 더 직관적으로 변했다.

### 명령

* 함수를 인수로 전달하는 기법을 사용하면 명령 디자인 패턴도 구현을 단순하게 만들 수 있다.
* 기본 개념은 명령 객체를 수신자와 호출자 사이에 놓고, 명령은 ```excute()```라는 단 하나의 메서드로 인터페이스를 구현한다.
* 이런 방식을 사용하면, 호출자는 수신자의 인터페이스를 알 필요가 없으며, 명령의 서브클래스를 통해 서로 다른 수신자를 추가할 수 있다.

In [68]:
class MacroCommand :
    """명령 리스트를 실행하는 명령"""
    
    def __init__(self, commands) :
        self.commands = list(commands)
    def __call__(self) :
        for command in self.commands :
            command()

* ```commands``` 인수로부터 리스트를 만들면 반복 가능한 객체임이 보장되며, 각각의 ```MecroCommand``` 객체 안에 명령에 대한 참조를 복사하게 된다.
* ```macroCommand``` 객체가 호출되면 ```self.commands```에 들어 있는 명령이 순서대로 호출된다.